<a href="https://colab.research.google.com/github/AbhishekAjoy/CIFAR10/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# Load Dataset

In [0]:
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# Scale Pixels

In [0]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# Model

In [0]:
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# Learning Curves

In [0]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [0]:
# load dataset
trainX, trainY, testX, testY = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
# define model
model = define_model()

In [7]:
# fit model
history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 14s 280us/step - loss: 2.0746 - accuracy: 0.3129 - val_loss: 1.5021 - val_accuracy: 0.4544
Epoch 2/100
50000/50000 [==============================] - 12s 232us/step - loss: 1.5532 - accuracy: 0.4348 - val_loss: 1.3462 - val_accuracy: 0.5129
Epoch 3/100
50000/50000 [==============================] - 12s 233us/step - loss: 1.4038 - accuracy: 0.4883 - val_loss: 1.2618 - val_accuracy: 0.5415
Epoch 4/100
50000/50000 [==============================] - 12s 233us/step - loss: 1.3051 - accuracy: 0.5265 - val_loss: 1.3755 - val_accuracy: 0.4914
Epoch 5/100
50000/50000 [==============================] - 12s 233us/step - loss: 1.2132 - accuracy: 0.5657 - val_loss: 1.1313 - val_accuracy: 0.5898
Epoch 6/100
50000/50000 [==============================] - 12s 235us/step - loss: 1.1476 - accuracy: 0.5895 - val_loss: 1.0603 - val_accuracy: 0.6142
Epoch 7/100
50000/50000 [=========================

In [0]:
model.save('final_model.h5')

In [9]:
# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)

> 84.520


# Load  Image

In [0]:
# load and prepare the image
filename = "sample_image.png"
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [0]:
def run_example():
	# load the image
  img = load_image('sample_image.png')
	# load model
  model = load_model('final_model.h5')
	# predict the class
  result = model.predict_classes(img)
  print('Prediction: ', end =" ")
  if result[0] == 0:
    print('airplane')
  elif result[0] == 1:
    print('automobile')
  elif result[0] == 2:
    print('bird')
  elif result[0] == 3:
    print('cat')
  elif result[0] == 4:
    print('deer')
  elif result[0] == 5:
    print('dog')
  elif result[0] == 6:
    print('frog')
  elif result[0] == 7:
    print('horse')
  elif result[0] == 8:
    print('ship')
  else:
    print('truck')

In [36]:
run_example()

Prediction:  airplane
